In [ ]:
!pip install langchain
!pip install langchain-community
!pip install openai

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('AI_KEY')

# 메모리 모듈의 종류

ConversationBufferMemory             
ConversationBufferWindowMemory    
ConversationTokenBufferMemory        
ConversationSummaryMemory           

## 1. ConversationBufferMemory    

- 이전 대화의 내용을 일정한 용량만큼(buffer) 기억해주는 모듈

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm_model = 'gpt-3.5-turbo-0301'
#llm을 정의
llm = ChatOpenAI(temperature=0.1, model=llm_model)

#대화한 내역을 기억하는 역할
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm,
    memory= memory,

    #동작하는 과정을 상세히 보여줄 지 여부
    verbose = True
)

In [ ]:
conversation.predict(input="Hi, my name is jeongeun")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is jeongeun
AI:

> Finished chain.


"Hello jeongeun, it's nice to meet you! My name is AI. How can I assist you today?"

In [ ]:
conversation.predict(input="What is 2 * 4?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is jeongeun
AI: Hello jeongeun, it's nice to meet you! My name is AI. How can I assist you today?
Human: What is 2 * 4?
AI:

> Finished chain.


'2 multiplied by 4 is equal to 8.'

In [ ]:
memory.buffer

"Human: Hi, my name is jeongeun\nAI: Hello jeongeun, it's nice to meet you! My name is AI. How can I assist you today?\nHuman: What is 2 * 4?\nAI: 2 multiplied by 4 is equal to 8."

In [ ]:
#기억 조작 : 우리가 나누지 않은 대화의 내용을 save_context라는 함수를 통해 전달
memory = ConversationBufferMemory()
memory.save_context({"input" : "Hi"},
    {"outputs" : "What's up?"})

In [ ]:
print(memory.buffer)

Human: Hi
AI: What's up?


### 2. conversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
#k의 크기에 따라, 기억하는 대화의 크기가 변화됨
memory = ConversationBufferWindowMemory(k=2)

In [ ]:
memory.save_context({"input" : "Hi"},
    {"outputs" : "What's up?"})
memory.save_context({"input" : "I'm good. How are you?"},
    {"outputs" : "Not bad"})

In [ ]:
print(memory.buffer)

Human: Hi
AI: What's up?
Human: I'm good. How are you?
AI: Not bad


### 3. conversationTokenBufferMemory     

- 토큰을 저장하도록 하는 메모리     
- LLM 사용 비용에 영향을 미치는(토큰으로 사용량 계산) 토큰을 얼마만큼 저장할건지 컨트롤 할 수 있게 하여, 비용 절감에 활용할 수 있음


In [ ]:
#랭체인, llm에서 토큰을 다루는 라이브러리
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.7 MB/s eta 0:00:00


In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

#"gpt-3.5-turbo-0301" : 응답해주는 llm모델의 버전
llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-0301")

In [ ]:
#옵션
#llm = 어떤 llm인가
#max_token_limit ===== 얼마만큼의 토큰을 기억할거냐?
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

memory.save_context({"input" : "Hi"},
    {"outputs" : "What's up?"})
memory.save_context({"input" : "I'm good. How are you?"},
    {"outputs" : "Not bad"})
memory.save_context({"input" : "I like reading books."},
    {"outputs" : "That's sounds interesting"})

In [ ]:
memory.buffer

"Human: I'm good. How are you?\nAI: Not bad\nHuman: I like reading books.\nAI: That's sounds interesting"

### 4. ConversationSummaryMemory     

- 지금까지의 대화 내용을 "요약"해서 기억하도록 함

In [ ]:
from langchain.memory import ConversationSummaryMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

In [ ]:
memory = ConversationSummaryMemory(llm=llm, max_token_limit=500)

memory.save_context({"input" : "Hi"},
    {"outputs" : "What's up?"})
memory.save_context({"input" : "I'm good. How are you?"},
    {"outputs" : "Not bad"})
memory.save_context({"input" : "What's the schedule today?"},
                    {"output": f"{schedule}"})

In [ ]:
memory.buffer

'The AI greets the human and provides a detailed schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in the latest in AI. The human asks about what is needed for a good demo show, and the AI explains that it requires a clear understanding of the audience and their needs, a well-prepared and rehearsed presentation, a clear and concise message, visual aids, and a thorough knowledge of the product or service. The human then asks about the requirements for the 8:00 am meeting with the product team, and the AI suggests bringing notes, relevant documents, and having a clear agenda and goals for the meeting.'

In [ ]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
conversation.predict(input="What is needed for meeting at 8:00 am?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The AI greets the human and provides a detailed schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in the latest in AI. The human asks about what is needed for a good demo show, and the AI explains that it requires a clear understanding of the audience and their needs, a well-prepared and rehearsed presentation, a clear and concise message, visual aids, and a thorough knowledge of the product or service.
Human: What is needed for meeting at 8:00 am?
AI:

> Finished chain.


'The meeting with the product team at 8:00 am will require you to bring your notes and any relevant documents or materials related to the project. It may also be helpful to have a clear agenda and goals for the meeting.'